**CURSO**: *Machine Learning* en Geociencias<br />
**Profesor**: Edier Aristizábal (evaristizabalg@unal.edu.co) <br />
**Classroom code**: [wv4cglx]

# 08: Validación Cruzada

## *train-test-split*

Este algortimo es muy rapido y es ideal para grandes bases de datos en donde los datos de entrenamiento y validacion son lo suficientemente represetativos del problema. Debido a que es rapido se puede utilizar cona algoritmos complejos y lento para el entrenamiento. Una falencia del método es que puede generar alta varianza debido a grandes diferencias entre los datos de entrenamiento y validación.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import warnings
warnings.simplefilter("ignore")

In [2]:
from sklearn.datasets import load_boston
X,y=load_boston(return_X_y=True)

Un agumento importante de este algoritmo es *random_state*, el cual permite obtener con el mismo numero la misma partición de datos aleatoria, para asegurar resultados similares. A continuacion se va a generar tres particiones, donde dos de ellas tiene el mismo valor semilla (1).

In [29]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,y, train_size=0.5, random_state=1)
X2_train,X2_test, y2_train,y2_test = train_test_split(X,y, random_state=1)
X3_train,X3_test, y3_train,y3_test = train_test_split(X,y, random_state=2)

In [4]:
print('Dimensiones de la matriz para entrenar:',X_train.shape)
print('Dimensiones del vector para entrenar:',y_train.shape)
print('Dimensiones de la matriz para validar:',X_test.shape)
print('Dimensiones del vector para validar:',y_test.shape)

Dimensiones de la matriz para entrenar: (379, 13)
Dimensiones del vector para entrenar: (379,)
Dimensiones de la matriz para validar: (127, 13)
Dimensiones del vector para validar: (127,)


Por defecto la función *train_test_split* divide la base de datos en 75% para entrenamiento y 25% para validación. pero cone l argumento *test_size* se puede especificar otro valor para el tamno de lso datso de validacion entre 0 y 1.

In [17]:
np.array_equal(X_train,X2_train)

True

Como se puede observar para los conjuntos con valor 1 las bases de datso aleatroias seleccioandas son exactamente iguales. En el caso doden se comapra con la seleccion aleatoria pero con semilla 2, las bases de datso no son iguales.

In [6]:
np.array_equal(X_train,X3_train)

False

A continuacion se puede implementar el modelo, en este caso Lasso, entrenarlo directamente y preguntar por el *score*.

In [8]:
LinearRegression().fit(X_train,y_train).score(X_test,y_test)

0.7789410172622834

## Validación cruzada (*cross validation*)

### *K-fold*

El método de *K-fold Cross Validation* permite obtener el desempeño del algoritmo con menor varianza que un particion sencilla de *train-test set split*. Este metodo divide lso datos en un número de K subconjuntos (k = 5 ó k = 10). Cada partición es denominada un *fold*. El algoritmo es entonces entrenado con K-1 subconjuntos y un subconjunto es utilizado para validar. Esto es k veces repetido por lo que se obtienen k valores de *score*. El algoritmo es por lo tanto entrenado y evaluado múltiples veces. Como resultado de esta función no se obtiene un modelo, ya que varios modelos son creados internamente, el propósito es sólamente evaluar que tan bien un algoritmo determinado va a generalizar con otros datos diferentes al entrenamiento.

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [40]:
kfold = KFold(n_splits=7, shuffle= True,random_state=1)
model = LinearRegression()
results = cross_val_score(model, X, y, cv=kfold, scoring='r2')
print(results)
print(results.mean())
print(results.std())

[0.76202909 0.8288083  0.4883423  0.82375273 0.58676533 0.74116246
 0.70983959]
0.7058142574151517
0.11645325435580706


A continuación se presenta la función *cross_validate*, la cual difiere de *cross_val_score* ya que permite definir múltiples métricas para estimar el ajuste, adicionalmente las salidas de la función son diferentes como se observa a continuación. Por defecto la función *cross_val_score* genera 3 particiones.

In [ ]:
from sklearn.model_selection import cross_validate

In [9]:
results_ridge = cross_validate(Ridge(),X,y,return_train_score=True,cv=5)
results_ridge

{'fit_time': array([0.29299998, 0.00199986, 0.00199986, 0.00099993, 0.00099993]),
 'score_time': array([0.00300002, 0.00099993, 0.00100017, 0.00100017, 0.00099993]),
 'test_score': array([ 0.66089569,  0.74094893,  0.62923672,  0.08530169, -0.17029513]),
 'train_score': array([0.74372716, 0.72395587, 0.68988726, 0.84024816, 0.73384871])}

In [10]:
test_scores = results_ridge['test_score']
train_scores = results_ridge['train_score']
print('Train scores:', np.mean(train_scores))
print('Test scores:', np.mean(test_scores))

Train scores: 0.746333431797385
Test scores: 0.38921758241023985


In [11]:
results_lasso = cross_validate(Lasso(),X,y,return_train_score=True,cv=5)
results_lasso

{'fit_time': array([0.10900021, 0.00200033, 0.00199986, 0.00099993, 0.00200009]),
 'score_time': array([0.00099993, 0.00099993, 0.00099993, 0.00099993, 0.00099993]),
 'test_score': array([0.56156843, 0.63385562, 0.33456629, 0.35466066, 0.27459294]),
 'train_score': array([0.69205313, 0.66722484, 0.62206251, 0.77992825, 0.68385778])}

In [12]:
test_scores = results_lasso['test_score']
train_scores = results_lasso['train_score']
print('Train scores:', np.mean(train_scores))
print('Test scores:', np.mean(test_scores))

Train scores: 0.6890252995484658
Test scores: 0.431848787926522


### *Stratified Kfold*

En problemas de regresión scikit-learn utiliza por defecto el k-fold, pero para problemas de clasificación scikit-learn utiliza *stratified k-fold cross-validation*, en donde los datos son divididos en igual proporción de las clases en la totalidad de datos, es decir preservando el porcentaje de observaciones en cada clase. Por esta razón es un buena estrategia para datos imbalanceados.

In [18]:
from sklearn.model_selection import StratifiedKFold

data= pd.read_excel('G:\My Drive\ANALISIS ESPACIAL APLICADO\datos\Cuencas_torrencialidad.xlsx', sheet_name='Hoja2')
X=data.drop(['Name', 'Flash flood record'],axis=1)
y=data['Flash flood record']

skfold=StratifiedKFold(n_splits=5)
results = cross_val_score(model, X, y, cv=skfold, scoring='r2')
print(results.mean())
print(results.std())

0.9381785777147649
0.018696245922007717


### Leave One Out Cross Validation
Un caso especial de *K-fold cross validation* es donde k sea igual al número de observaciones. Este tipo de variación se denomina *leave-one-out cross validation*.

In [13]:
from sklearn.model_selection import LeaveOneOut

In [16]:
loocv = LeaveOneOut()
model = LinearRegression()
results = cross_val_score(model, X, y, cv=loocv, scoring='r2')
print(results.mean())
print(results.std())

nan
nan


### ShuffleSplit

Otra variación de *k-fold* es generar una partición aleatoria como *train-test-split*, pero repite el proceso de partición y evaluación múltiples veces como *K-fold*.

In [45]:
from sklearn.model_selection import ShuffleSplit

In [56]:
kfold = ShuffleSplit(n_splits=5, test_size=0.3, random_state=1)
model = LinearRegression()
results = cross_val_score(model, X, y, cv=kfold, scoring='r2')
print(results.mean())
print(results.std())

0.7288121678315397
0.049847359292500816
